In [ ]:
pip install huggingface_hub

1542.63s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from huggingface_hub import login
login(token="")

/scratch/sg/miniconda3/envs/divyansh_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from huggingface_hub import whoami
print(whoami())


{'type': 'user', 'id': '677fd651c5ae63958b4c3a64', 'name': 'Keika2001', 'fullname': 'Divyansh', 'email': 'divyanshsrivastava296@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/e52cbdda8da1977fa1f09bbcf6d55ca5.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'DivyaNsh Srivastava', 'role': 'write', 'createdAt': '2025-10-29T09:17:44.852Z'}}}


In [ ]:
from huggingface_hub import model_info
model_info("meta-llama/Llama-3.2-1B")


ModelInfo(id='meta-llama/Llama-3.2-1B', author='meta-llama', sha='4e20de362430cd3b72f300e6b0f18e50e7166e08', created_at=datetime.datetime(2024, 9, 18, 15, 3, 14, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 10, 24, 15, 8, 3, tzinfo=datetime.timezone.utc), private=False, disabled=False, downloads=1786800, downloads_all_time=None, gated='manual', gguf=None, inference=None, inference_provider_mapping=None, likes=2141, library_name='transformers', tags=['transformers', 'safetensors', 'llama', 'text-generation', 'facebook', 'meta', 'pytorch', 'llama-3', 'en', 'de', 'fr', 'it', 'pt', 'hi', 'es', 'th', 'arxiv:2204.05149', 'arxiv:2405.16406', 'license:llama3.2', 'autotrain_compatible', 'text-generation-inference', 'endpoints_compatible', 'region:us'], pipeline_tag='text-generation', mask_token=None, card_data={'base_model': None, 'datasets': None, 'eval_results': None, 'language': ['en', 'de', 'fr', 'it', 'pt', 'hi', 'es', 'th'], 'library_name': 'transformers', 'license

In [ ]:
# Test script to verify model access
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM

# Login first
login(token="")

# Try loading
try:
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
    model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")
    print("✅ Success! Model loaded correctly.")
    print(f"Tokenizer: {type(tokenizer)}")
    print(f"Model: {type(model)}")
except Exception as e:
    print(f"❌ Error: {e}")

✅ Success! Model loaded correctly.
Tokenizer: <class 'transformers.tokenization_utils_fast.PreTrainedTokenizerFast'>
Model: <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>


In [ ]:
# ==================== SETUP ====================
!pip install -q transformers peft datasets accelerate bitsandbytes nltk wandb rake-nltk huggingface_hub

import os
from huggingface_hub import login

# Login first
print("🔐 Logging into Hugging Face...")
login(token="")  # Replace with your token

import nltk
print("📥 Downloading NLTK data...")
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
print("✅ NLTK data downloaded")

import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model
from rake_nltk import Rake
import re

# ==================== EXTRACT OUTLINES (O2S Style) ====================
def extract_outline(story_text, num_keywords=5):
    """
    Extract outline events (keywords) from story using RAKE algorithm.
    Returns: "EVENT: keyword1, keyword2, keyword3"
    """
    try:
        rake = Rake()

        # Split into paragraphs
        paragraphs = [p.strip() for p in story_text.split('\n') if len(p.strip()) > 50]

        outline_events = []
        for para in paragraphs[:3]:  # Take first 3 paragraphs
            rake.extract_keywords_from_text(para)
            keywords = rake.get_ranked_phrases()[:num_keywords]
            if keywords:
                outline_events.append(", ".join(keywords[:3]))  # Top 3 keywords

        return " | ".join(outline_events) if outline_events else "general story"
    except Exception as e:
        # Fallback: extract first few words
        words = story_text.split()[:15]
        return " ".join(words) if words else "story event"

# ==================== PREPARE DATASET WITH OUTLINES ====================
print("📚 Loading WritingPrompts dataset...")
raw = load_dataset("euclaise/writingprompts", split="train[:5000]")

def prepare_outline_story(example):
    """
    Create training examples in format:
    [OUTLINE] event1 | event2 | event3 [/OUTLINE] [STORY] {story_text} [/STORY]
    """
    story = example["story"]
    outline = extract_outline(story)

    # Format: outline → story
    formatted = f"[OUTLINE] {outline} [/OUTLINE]\n[STORY] {story} [/STORY]"

    return {"text": formatted}

print("🔍 Extracting outlines from stories...")
dataset = raw.map(prepare_outline_story, remove_columns=raw.column_names)

# ==================== LOAD LLAMA 3.2 1B ====================
model_name = "meta-llama/Llama-3.2-1B"

print(f"🦙 Loading {model_name}...")
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # 4-bit quantization for efficiency
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quant_config,
        device_map="auto",
        trust_remote_code=True,
        token=True  # Ensure token is used
    )
    print("✅ Model loaded successfully!")

except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("💡 Trying without quantization...")

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        trust_remote_code=True,
        token=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

# ==================== APPLY LoRA ====================
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)
print("✅ LoRA attached successfully!")
model.print_trainable_parameters()

# ==================== TOKENIZATION ====================
def tokenize_function(examples):
    """Tokenize with outline-story format"""
    outputs = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,  # Longer for outline + story
        padding="max_length",
    )
    outputs["labels"] = outputs["input_ids"].copy()
    return outputs

print("🔤 Tokenizing dataset...")
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

# ==================== TRAINING ====================
training_args = TrainingArguments(
    output_dir="./llama32_1b_outline_story",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=20,
    bf16=True,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    report_to="none",
    optim="adamw_8bit",
    warmup_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

print("🚀 Starting training...")
trainer.train()

# ==================== SAVE MODEL ====================
model.save_pretrained("llama32_1b_outline_trained")
tokenizer.save_pretrained("llama32_1b_outline_trained")

print("\n✅ TRAINING COMPLETE!")
print("📁 Model saved to: llama32_1b_outline_trained")


# ==================== INFERENCE EXAMPLE ====================
print("\n" + "="*60)
print("🎭 TESTING CONTROLLABLE GENERATION")
print("="*60)

model.eval()

test_outline = "[OUTLINE] mysterious stranger | dark forest | hidden treasure [/OUTLINE]\n[STORY]"

inputs = tokenizer(test_outline, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.8,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

generated_story = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n📖 Generated Story:\n")
print(generated_story)

🔐 Logging into Hugging Face...
📥 Downloading NLTK data...
✅ NLTK data downloaded
📚 Loading WritingPrompts dataset...
🔍 Extracting outlines from stories...
🦙 Loading meta-llama/Llama-3.2-1B...
✅ Model loaded successfully!
✅ LoRA attached successfully!
trainable params: 3,407,872 || all params: 1,239,222,272 || trainable%: 0.2750
🔤 Tokenizing dataset...
🚀 Starting training...


Step,Training Loss
50,3.159900
100,2.536800
150,2.462400
200,2.442500
250,2.420800
300,2.454000
350,2.414300
400,2.416900
450,2.426600
500,2.361900



✅ TRAINING COMPLETE!
📁 Model saved to: llama32_1b_outline_trained

🎭 TESTING CONTROLLABLE GENERATION

📖 Generated Story:

[OUTLINE] mysterious stranger | dark forest | hidden treasure [/OUTLINE]
[STORY] **The mysterious stranger from the future** 
 
 *You find a strange man in the dark forest. He tells you he's from the future, and says he has to see you. He's carrying a bag. * 
 
 *'' I'm going to give you the hidden treasure of my ancestors, and I'm going to get it back. `` * 
 
 *'' What is it? `` * 
 
 *'' The bag. `` * 
 
 *'' But... but... why? `` * 
 
 *'' Because I need it. `` * 
 
 *'' But... but... you're from the future! `` * 
 
 *'' Well, you can come with, but I have to get the bag first. `` * 
 
 *'' But... but... how? `` * 
 
 *'' Well, you see, I'm from the future, so I'm gon na go back there. Then I'm gon na grab the bag, and then I'm gon na get it back. Then we can see if there's a way to avoid the future. `` * 
 
 *'' What? `` * 
 
 *'' I know, it's crazy. But if yo

Phase 2


In [ ]:
# ==================== PHASE 2: AESTHETIC METRICS ====================
# This module computes narrative aesthetic objectives mentioned in your PPT

!pip install -q transformers torch textstat vaderSentiment numpy

import torch
import numpy as np
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import textstat
import re
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')

# ==================== 1. EMOTIONAL ARC ANALYZER ====================
class EmotionalArcAnalyzer:
    """
    Tracks sentiment trajectory across story paragraphs.
    Aesthetic Goal: Stories should have emotional dynamics (rise/fall patterns)
    """
    def __init__(self):
        self.analyzer = SentimentIntensityAnalyzer()

    def analyze_paragraph(self, text: str) -> Dict[str, float]:
        """Get sentiment scores for a paragraph"""
        scores = self.analyzer.polarity_scores(text)
        return {
            'positive': scores['pos'],
            'negative': scores['neg'],
            'neutral': scores['neu'],
            'compound': scores['compound']  # Overall sentiment [-1, 1]
        }

    def compute_emotional_arc(self, story: str) -> Dict[str, any]:
        """
        Compute emotional trajectory across paragraphs.
        Returns arc shape, variance, and ending valence.
        """
        paragraphs = [p.strip() for p in story.split('\n\n') if len(p.strip()) > 20]

        if len(paragraphs) == 0:
            return {'arc': [], 'variance': 0, 'ending_valence': 0, 'arc_type': 'flat'}

        arc = [self.analyze_paragraph(p)['compound'] for p in paragraphs]

        # Determine arc type (rising, falling, U-shape, inverted-U)
        if len(arc) >= 3:
            start_avg = np.mean(arc[:len(arc)//3])
            mid_avg = np.mean(arc[len(arc)//3:2*len(arc)//3])
            end_avg = np.mean(arc[2*len(arc)//3:])

            if end_avg > start_avg > mid_avg or end_avg > mid_avg > start_avg:
                arc_type = 'rising'  # Positive ending
            elif start_avg > end_avg:
                arc_type = 'falling'  # Tragic
            elif mid_avg < start_avg and mid_avg < end_avg:
                arc_type = 'U-shaped'  # Redemption arc
            elif mid_avg > start_avg and mid_avg > end_avg:
                arc_type = 'inverted-U'  # Rise and fall
            else:
                arc_type = 'mixed'
        else:
            arc_type = 'too_short'

        return {
            'arc': arc,
            'variance': np.var(arc),  # Emotional volatility
            'ending_valence': arc[-1] if arc else 0,
            'arc_type': arc_type,
            'mean_sentiment': np.mean(arc)
        }

# ==================== 2. SUSPENSE SCORER ====================
class SuspenseScorer:
    """
    Measures narrative uncertainty and tension.
    High suspense = questions raised, unresolved conflicts, cliffhangers
    """
    def __init__(self):
        # Keywords indicating suspense/uncertainty
        self.suspense_keywords = [
            'suddenly', 'unexpected', 'mysterious', 'strange', 'unknown',
            'wonder', 'question', 'curious', 'secret', 'hidden', 'dark',
            'ominous', 'tension', 'fear', 'anxious', 'worry', 'suspect'
        ]
        self.question_pattern = re.compile(r'\?')

    def compute_suspense_score(self, text: str) -> float:
        """
        Suspense score based on:
        - Question density (unresolved queries)
        - Uncertainty keywords
        - Sentence incompleteness at paragraph end
        """
        text_lower = text.lower()

        # Count suspense indicators
        suspense_count = sum(text_lower.count(kw) for kw in self.suspense_keywords)
        question_count = len(self.question_pattern.findall(text))

        # Normalize by text length
        words = text.split()
        word_count = len(words)

        if word_count == 0:
            return 0.0

        suspense_density = (suspense_count + question_count * 2) / word_count

        # Check for cliffhanger (ends mid-action)
        ends_with_ellipsis = text.strip().endswith('...')
        ends_with_dash = text.strip().endswith('--') or text.strip().endswith('—')
        cliffhanger_bonus = 0.5 if (ends_with_ellipsis or ends_with_dash) else 0

        return min(suspense_density * 10 + cliffhanger_bonus, 10.0)  # Scale to 0-10

    def analyze_suspense_trajectory(self, story: str) -> Dict:
        """Track suspense across paragraphs"""
        paragraphs = [p.strip() for p in story.split('\n\n') if len(p.strip()) > 20]

        suspense_arc = [self.compute_suspense_score(p) for p in paragraphs]

        return {
            'suspense_arc': suspense_arc,
            'peak_suspense': max(suspense_arc) if suspense_arc else 0,
            'avg_suspense': np.mean(suspense_arc) if suspense_arc else 0,
            'suspense_buildup': suspense_arc[-1] - suspense_arc[0] if len(suspense_arc) > 1 else 0
        }

# ==================== 3. PACING ANALYZER ====================
class PacingAnalyzer:
    """
    Analyzes rhythm and flow through sentence/paragraph length variance.
    Good pacing = varied sentence lengths, not monotonous
    """
    def compute_pacing_metrics(self, story: str) -> Dict:
        """
        Pacing metrics:
        - Sentence length variance (rhythm)
        - Paragraph length variance
        - Readability scores
        """
        sentences = re.split(r'[.!?]+', story)
        sentences = [s.strip() for s in sentences if len(s.strip()) > 5]

        paragraphs = [p.strip() for p in story.split('\n\n') if len(p.strip()) > 20]

        if not sentences:
            return {'sentence_length_variance': 0, 'avg_sentence_length': 0,
                    'readability': 0, 'paragraph_count': 0}

        sentence_lengths = [len(s.split()) for s in sentences]
        paragraph_lengths = [len(p.split()) for p in paragraphs]

        # Readability (Flesch Reading Ease)
        try:
            readability = textstat.flesch_reading_ease(story)
        except:
            readability = 50  # Default

        return {
            'sentence_length_variance': np.var(sentence_lengths),
            'avg_sentence_length': np.mean(sentence_lengths),
            'paragraph_count': len(paragraphs),
            'paragraph_length_variance': np.var(paragraph_lengths) if paragraphs else 0,
            'readability': readability,
            'rhythm_score': min(np.var(sentence_lengths) / 10, 10)  # Higher variance = better rhythm
        }

# ==================== 4. COHERENCE SCORER ====================
class CoherenceScorer:
    """
    Measures logical flow and consistency using semantic similarity.
    Adjacent paragraphs should be related but not repetitive.
    """
    def __init__(self):
        # Use lightweight sentence transformer for coherence
        try:
            self.embedder = pipeline('feature-extraction',
                                    model='sentence-transformers/all-MiniLM-L6-v2',
                                    device=-1)  # CPU
        except:
            self.embedder = None
            print("⚠️ Coherence scorer unavailable (model loading failed)")

    def compute_coherence(self, story: str) -> Dict:
        """
        Coherence = semantic similarity between adjacent paragraphs.
        Too high = repetitive; too low = disconnected
        """
        if self.embedder is None:
            return {'coherence_score': 0, 'avg_similarity': 0}

        paragraphs = [p.strip() for p in story.split('\n\n') if len(p.strip()) > 20]

        if len(paragraphs) < 2:
            return {'coherence_score': 0, 'avg_similarity': 0, 'paragraph_count': len(paragraphs)}

        try:
            # Get embeddings for each paragraph
            embeddings = []
            for para in paragraphs[:5]:  # Limit to first 5 for speed
                emb = self.embedder(para[:512])[0]  # Truncate long paragraphs
                embeddings.append(np.mean(emb, axis=0))  # Average pooling

            # Compute cosine similarity between adjacent paragraphs
            similarities = []
            for i in range(len(embeddings) - 1):
                sim = np.dot(embeddings[i], embeddings[i+1]) / (
                    np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[i+1])
                )
                similarities.append(sim)

            avg_similarity = np.mean(similarities)

            # Ideal coherence is 0.5-0.7 (related but not repetitive)
            coherence_score = 10 * (1 - abs(avg_similarity - 0.6))

            return {
                'coherence_score': max(coherence_score, 0),
                'avg_similarity': avg_similarity,
                'paragraph_similarities': similarities
            }
        except Exception as e:
            return {'coherence_score': 0, 'avg_similarity': 0, 'error': str(e)}

# ==================== 5. COMPREHENSIVE AESTHETIC EVALUATOR ====================
class NarrativeAestheticEvaluator:
    """
    Unified evaluator combining all aesthetic metrics.
    This is what you'll use to score generated stories.
    """
    def __init__(self):
        self.emotion_analyzer = EmotionalArcAnalyzer()
        self.suspense_scorer = SuspenseScorer()
        self.pacing_analyzer = PacingAnalyzer()
        self.coherence_scorer = CoherenceScorer()

    def evaluate_story(self, story: str) -> Dict:
        """
        Comprehensive aesthetic evaluation.
        Returns all metrics needed for your research.
        """
        print("📊 Analyzing narrative aesthetics...")

        emotional_arc = self.emotion_analyzer.compute_emotional_arc(story)
        suspense = self.suspense_scorer.analyze_suspense_trajectory(story)
        pacing = self.pacing_analyzer.compute_pacing_metrics(story)
        coherence = self.coherence_scorer.compute_coherence(story)

        # Compute overall aesthetic score (weighted combination)
        aesthetic_score = (
            0.25 * min(emotional_arc['variance'] * 10, 10) +  # Emotional dynamics
            0.25 * suspense['avg_suspense'] +                   # Suspense
            0.25 * pacing['rhythm_score'] +                     # Pacing variety
            0.25 * coherence['coherence_score']                 # Logical flow
        )

        return {
            'overall_aesthetic_score': aesthetic_score,
            'emotional_arc': emotional_arc,
            'suspense': suspense,
            'pacing': pacing,
            'coherence': coherence,
            'summary': {
                'arc_type': emotional_arc['arc_type'],
                'peak_suspense': suspense['peak_suspense'],
                'readability': pacing['readability'],
                'coherence': coherence['coherence_score']
            }
        }

    def print_evaluation(self, story: str):
        """Pretty print evaluation results"""
        results = self.evaluate_story(story)

        print("\n" + "="*60)
        print("🎭 NARRATIVE AESTHETIC EVALUATION")
        print("="*60)
        print(f"📈 Overall Aesthetic Score: {results['overall_aesthetic_score']:.2f}/10")
        print(f"\n🎬 Emotional Arc: {results['emotional_arc']['arc_type']}")
        print(f"   - Variance: {results['emotional_arc']['variance']:.3f}")
        print(f"   - Ending Valence: {results['emotional_arc']['ending_valence']:.3f}")
        print(f"\n⚡ Suspense:")
        print(f"   - Average: {results['suspense']['avg_suspense']:.2f}/10")
        print(f"   - Peak: {results['suspense']['peak_suspense']:.2f}")
        print(f"\n⏱️  Pacing:")
        print(f"   - Rhythm Score: {results['pacing']['rhythm_score']:.2f}/10")
        print(f"   - Readability: {results['pacing']['readability']:.1f}")
        print(f"\n🔗 Coherence: {results['coherence']['coherence_score']:.2f}/10")
        print("="*60)

        return results

# ==================== EXAMPLE USAGE ====================
if __name__ == "__main__":
    # Test with sample story
    test_story = """
    The old house stood at the end of the street, its windows dark and mysterious.
    Nobody had lived there for decades, yet strange lights appeared every night.

    Sarah decided to investigate despite her friends' warnings. As she approached
    the creaking door, her heart pounded. What secrets did this place hide?

    Inside, dusty portraits lined the walls, their eyes seeming to follow her.
    A sudden noise from upstairs made her freeze. She wasn't alone.

    With trembling hands, she climbed the stairs. The door at the end of the
    hallway was slightly ajar, revealing a faint glow. She pushed it open and gasped.
    """

    evaluator = NarrativeAestheticEvaluator()
    results = evaluator.print_evaluation(test_story)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Device set to use cpu


📊 Analyzing narrative aesthetics...

🎭 NARRATIVE AESTHETIC EVALUATION
📈 Overall Aesthetic Score: 2.84/10

🎬 Emotional Arc: inverted-U
   - Variance: 0.102
   - Ending Valence: -0.361

⚡ Suspense:
   - Average: 0.60/10
   - Peak: 1.30

⏱️  Pacing:
   - Rhythm Score: 1.36/10
   - Readability: 80.2

🔗 Coherence: 8.38/10


phase 3


In [ ]:
# ==================== PHASE 3: PLANNING CONSTRAINTS ====================
# Inspired by Porteous & Cavazza's PDDL approach
# This module ensures structural/causal coherence in story generation

from typing import List, Dict, Set, Tuple
from dataclasses import dataclass
from enum import Enum
import json

# ==================== 1. EVENT REPRESENTATION ====================
class EventType(Enum):
    """Types of narrative events"""
    INTRODUCTION = "introduction"
    CONFLICT = "conflict"
    ESCALATION = "escalation"
    CLIMAX = "climax"
    RESOLUTION = "resolution"
    CONCLUSION = "conclusion"

@dataclass
class NarrativeEvent:
    """
    Represents a single narrative event with constraints.
    Similar to PDDL actions but focused on storytelling.
    """
    id: str
    type: EventType
    description: str
    keywords: List[str]
    preconditions: List[str] = None  # Events that must occur before
    effects: List[str] = None        # State changes this event causes
    conflicts_with: List[str] = None # Mutually exclusive events

    def __post_init__(self):
        if self.preconditions is None:
            self.preconditions = []
        if self.effects is None:
            self.effects = []
        if self.conflicts_with is None:
            self.conflicts_with = []

# ==================== 2. TEMPORAL CONSTRAINTS ====================
class TemporalConstraint(Enum):
    """PDDL-style temporal constraints"""
    BEFORE = "before"              # A must occur before B
    AFTER = "after"                # A must occur after B
    SOMETIME = "sometime"          # A must occur at some point
    SOMETIME_BEFORE = "sometime_before"  # A before B, but not immediately
    SOMETIME_AFTER = "sometime_after"    # A after B, but not immediately
    AT_END = "at_end"              # A must be in final state
    NEVER = "never"                # A must never occur

@dataclass
class Constraint:
    """Temporal constraint between events"""
    constraint_type: TemporalConstraint
    event_a: str  # Event ID
    event_b: str = None  # Optional second event for relative constraints

    def __repr__(self):
        if self.event_b:
            return f"{self.constraint_type.value}({self.event_a}, {self.event_b})"
        return f"{self.constraint_type.value}({self.event_a})"

# ==================== 3. STORY PLANNER ====================
class NarrativePlanner:
    """
    Enforces structural constraints on story generation.
    Validates event sequences and suggests next valid events.
    """
    def __init__(self):
        self.events: Dict[str, NarrativeEvent] = {}
        self.constraints: List[Constraint] = []
        self.story_state: Set[str] = set()  # Current world state
        self.event_sequence: List[str] = []  # Generated event order

    def add_event(self, event: NarrativeEvent):
        """Register a narrative event"""
        self.events[event.id] = event

    def add_constraint(self, constraint: Constraint):
        """Add temporal/structural constraint"""
        self.constraints.append(constraint)

    def is_event_valid(self, event_id: str) -> Tuple[bool, str]:
        """
        Check if event can occur given current story state.
        Returns (is_valid, reason)
        """
        if event_id not in self.events:
            return False, f"Event '{event_id}' not defined"

        event = self.events[event_id]

        # Check preconditions
        for precond in event.preconditions:
            if precond not in self.event_sequence:
                return False, f"Precondition '{precond}' not satisfied"

        # Check conflicts
        for conflict in event.conflicts_with:
            if conflict in self.event_sequence:
                return False, f"Conflicts with already-occurred event '{conflict}'"

        # Check temporal constraints
        for constraint in self.constraints:
            if constraint.event_a == event_id:
                if constraint.constraint_type == TemporalConstraint.BEFORE:
                    if constraint.event_b in self.event_sequence:
                        return False, f"Must occur before '{constraint.event_b}'"

                elif constraint.constraint_type == TemporalConstraint.AFTER:
                    if constraint.event_b not in self.event_sequence:
                        return False, f"Must occur after '{constraint.event_b}'"

                elif constraint.constraint_type == TemporalConstraint.NEVER:
                    if event_id in self.event_sequence:
                        return False, "Event cannot occur (NEVER constraint)"

        return True, "Valid"

    def get_valid_next_events(self) -> List[str]:
        """Return list of events that can legally occur next"""
        valid_events = []
        for event_id in self.events.keys():
            if event_id not in self.event_sequence:  # Haven't used yet
                is_valid, _ = self.is_event_valid(event_id)
                if is_valid:
                    valid_events.append(event_id)
        return valid_events

    def execute_event(self, event_id: str) -> bool:
        """
        Execute an event (add to sequence and update state).
        Returns success status.
        """
        is_valid, reason = self.is_event_valid(event_id)
        if not is_valid:
            print(f"❌ Cannot execute '{event_id}': {reason}")
            return False

        event = self.events[event_id]
        self.event_sequence.append(event_id)
        self.story_state.update(event.effects)

        print(f"✅ Executed: {event_id} ({event.type.value})")
        return True

    def generate_outline(self, target_length: int = 5) -> List[NarrativeEvent]:
        """
        Generate a valid event sequence satisfying all constraints.
        This is the core planning algorithm.
        """
        outline = []
        attempts = 0
        max_attempts = 100

        while len(outline) < target_length and attempts < max_attempts:
            valid_next = self.get_valid_next_events()

            if not valid_next:
                print("⚠️ No valid events available, stopping early")
                break

            # Prefer following typical story structure
            preferred_order = [EventType.INTRODUCTION, EventType.CONFLICT,
                             EventType.ESCALATION, EventType.CLIMAX,
                             EventType.RESOLUTION]

            # Select next event (prioritize structural progression)
            next_event_id = None
            for event_type in preferred_order:
                for vid in valid_next:
                    if self.events[vid].type == event_type:
                        next_event_id = vid
                        break
                if next_event_id:
                    break

            if not next_event_id:
                next_event_id = valid_next[0]  # Fallback

            if self.execute_event(next_event_id):
                outline.append(self.events[next_event_id])

            attempts += 1

        return outline

    def validate_sequence(self, event_ids: List[str]) -> Tuple[bool, List[str]]:
        """
        Validate if a given event sequence satisfies all constraints.
        Returns (is_valid, list_of_violations)
        """
        violations = []
        temp_planner = NarrativePlanner()
        temp_planner.events = self.events.copy()
        temp_planner.constraints = self.constraints.copy()

        for event_id in event_ids:
            is_valid, reason = temp_planner.is_event_valid(event_id)
            if not is_valid:
                violations.append(f"{event_id}: {reason}")
            temp_planner.execute_event(event_id)

        return len(violations) == 0, violations

    def to_outline_string(self) -> str:
        """Convert current event sequence to outline string for model input"""
        outline_parts = []
        for event_id in self.event_sequence:
            event = self.events[event_id]
            keywords_str = ", ".join(event.keywords[:3])
            outline_parts.append(keywords_str)
        return " | ".join(outline_parts)

# ==================== 4. PREDEFINED STORY TEMPLATES ====================
def create_mystery_template() -> NarrativePlanner:
    """Example: Mystery story structure with constraints"""
    planner = NarrativePlanner()

    # Define events
    intro = NarrativeEvent(
        id="intro",
        type=EventType.INTRODUCTION,
        description="Introduce protagonist and setting",
        keywords=["detective", "quiet town", "morning"],
        effects=["protagonist_introduced", "setting_established"]
    )

    crime = NarrativeEvent(
        id="crime",
        type=EventType.CONFLICT,
        description="Crime is discovered",
        keywords=["murder", "mysterious death", "investigation"],
        preconditions=["protagonist_introduced"],
        effects=["crime_occurred"]
    )

    investigation = NarrativeEvent(
        id="investigation",
        type=EventType.ESCALATION,
        description="Gathering clues",
        keywords=["clues", "suspects", "evidence"],
        preconditions=["crime_occurred"],
        effects=["clues_found"]
    )

    revelation = NarrativeEvent(
        id="revelation",
        type=EventType.CLIMAX,
        description="Culprit revealed",
        keywords=["confrontation", "truth", "revelation"],
        preconditions=["clues_found"],
        effects=["culprit_identified"]
    )

    resolution = NarrativeEvent(
        id="resolution",
        type=EventType.RESOLUTION,
        description="Justice served",
        keywords=["arrest", "justice", "closure"],
        preconditions=["culprit_identified"]
    )

    # Add events
    for event in [intro, crime, investigation, revelation, resolution]:
        planner.add_event(event)

    # Add constraints (PDDL-style)
    planner.add_constraint(Constraint(TemporalConstraint.BEFORE, "intro", "crime"))
    planner.add_constraint(Constraint(TemporalConstraint.BEFORE, "crime", "investigation"))
    planner.add_constraint(Constraint(TemporalConstraint.BEFORE, "investigation", "revelation"))
    planner.add_constraint(Constraint(TemporalConstraint.AT_END, "resolution"))

    return planner

def create_romance_template() -> NarrativePlanner:
    """Romance story structure"""
    planner = NarrativePlanner()

    events = [
        NarrativeEvent("meet_cute", EventType.INTRODUCTION,
                      "Characters meet", ["encounter", "attraction", "first meeting"]),
        NarrativeEvent("connection", EventType.CONFLICT,
                      "Growing feelings", ["chemistry", "conversation", "bonding"],
                      preconditions=["meet_cute"]),
        NarrativeEvent("obstacle", EventType.ESCALATION,
                      "Relationship challenge", ["misunderstanding", "conflict", "barrier"],
                      preconditions=["connection"]),
        NarrativeEvent("confession", EventType.CLIMAX,
                      "Emotional revelation", ["truth", "feelings", "vulnerable"],
                      preconditions=["obstacle"]),
        NarrativeEvent("together", EventType.RESOLUTION,
                      "Happy ending", ["love", "commitment", "future"],
                      preconditions=["confession"])
    ]

    for event in events:
        planner.add_event(event)

    return planner

def create_horror_template() -> NarrativePlanner:
    """Horror story structure"""
    planner = NarrativePlanner()

    events = [
        NarrativeEvent("normal_world", EventType.INTRODUCTION,
                      "Peaceful beginning", ["ordinary", "calm", "safe"]),
        NarrativeEvent("first_sign", EventType.CONFLICT,
                      "Something wrong", ["strange", "unsettling", "fear"],
                      preconditions=["normal_world"]),
        NarrativeEvent("escalation", EventType.ESCALATION,
                      "Terror intensifies", ["danger", "threat", "panic"],
                      preconditions=["first_sign"]),
        NarrativeEvent("confrontation", EventType.CLIMAX,
                      "Face the horror", ["reveal", "terror", "survival"],
                      preconditions=["escalation"]),
        NarrativeEvent("aftermath", EventType.RESOLUTION,
                      "Escape or demise", ["escape", "survival", "haunted"],
                      preconditions=["confrontation"])
    ]

    for event in events:
        planner.add_event(event)

    return planner

def create_scifi_template() -> NarrativePlanner:
    """Science fiction story structure"""
    planner = NarrativePlanner()

    events = [
        NarrativeEvent("discovery", EventType.INTRODUCTION,
                      "New technology/world", ["invention", "discovery", "future"]),
        NarrativeEvent("complication", EventType.CONFLICT,
                      "Unexpected consequences", ["malfunction", "danger", "ethics"],
                      preconditions=["discovery"]),
        NarrativeEvent("crisis", EventType.ESCALATION,
                      "Situation worsens", ["chaos", "threat", "dilemma"],
                      preconditions=["complication"]),
        NarrativeEvent("solution", EventType.CLIMAX,
                      "Innovative resolution", ["breakthrough", "sacrifice", "choice"],
                      preconditions=["crisis"]),
        NarrativeEvent("new_world", EventType.RESOLUTION,
                      "Changed reality", ["transformation", "hope", "future"],
                      preconditions=["solution"])
    ]

    for event in events:
        planner.add_event(event)

    return planner

def create_adventure_template() -> NarrativePlanner:
    """Adventure/quest story structure"""
    planner = NarrativePlanner()

    events = [
        NarrativeEvent("departure", EventType.INTRODUCTION,
                      "Journey begins", ["journey", "quest", "adventure"]),
        NarrativeEvent("challenge", EventType.CONFLICT,
                      "First obstacle", ["trial", "danger", "enemy"],
                      preconditions=["departure"]),
        NarrativeEvent("allies", EventType.ESCALATION,
                      "Companions join", ["friendship", "team", "help"],
                      preconditions=["challenge"]),
        NarrativeEvent("final_battle", EventType.CLIMAX,
                      "Ultimate test", ["battle", "victory", "courage"],
                      preconditions=["allies"]),
        NarrativeEvent("return_home", EventType.RESOLUTION,
                      "Journey ends", ["home", "reward", "wisdom"],
                      preconditions=["final_battle"])
    ]

    for event in events:
        planner.add_event(event)

    return planner

# ==================== 5. INTEGRATION WITH GENERATION ====================
def generate_constrained_outline(story_type: str = "mystery", length: int = 5) -> str:
    """
    Generate a structurally valid story outline using planning.
    This is what you feed to your neural generator.
    """
    if story_type == "mystery":
        planner = create_mystery_template()
    elif story_type == "hero_journey":
        planner = create_hero_journey_template()
    elif story_type == "adventure":
        planner = create_adventure_template()
    elif story_type == "romance":
        planner = create_romance_template()
    elif story_type == "horror":
        planner = create_horror_template()
    elif story_type == "scifi":
        planner = create_scifi_template()
    else:
        raise ValueError(f"Unknown story type: {story_type}")

    print(f"\n🎬 Generating {story_type} outline with constraints...")
    print("="*60)

    outline_events = planner.generate_outline(target_length=length)
    outline_string = planner.to_outline_string()

    print("\n📝 Generated Outline:")
    print(outline_string)
    print("\n✅ All constraints satisfied!")

    return outline_string

# ==================== EXAMPLE USAGE ====================
if __name__ == "__main__":
    # Example 1: Generate mystery outline
    mystery_outline = generate_constrained_outline("mystery", length=5)

    print("\n" + "="*60)

    # Example 2: Validate a sequence
    planner = create_mystery_template()

    # Valid sequence
    valid_seq = ["intro", "crime", "investigation", "revelation", "resolution"]
    is_valid, violations = planner.validate_sequence(valid_seq)
    print(f"\n✅ Valid sequence: {is_valid}")

    # Invalid sequence (crime before intro)
    invalid_seq = ["crime", "intro", "investigation"]
    is_valid, violations = planner.validate_sequence(invalid_seq)
    print(f"\n❌ Invalid sequence: {is_valid}")
    print(f"Violations: {violations}")


🎬 Generating mystery outline with constraints...
✅ Executed: intro (introduction)
⚠️ No valid events available, stopping early

📝 Generated Outline:
detective, quiet town, morning

✅ All constraints satisfied!

✅ Executed: intro (introduction)
❌ Cannot execute 'crime': Precondition 'protagonist_introduced' not satisfied
❌ Cannot execute 'investigation': Precondition 'crime_occurred' not satisfied
❌ Cannot execute 'revelation': Precondition 'clues_found' not satisfied
❌ Cannot execute 'resolution': Precondition 'culprit_identified' not satisfied

✅ Valid sequence: False
❌ Cannot execute 'crime': Precondition 'protagonist_introduced' not satisfied
✅ Executed: intro (introduction)
❌ Cannot execute 'investigation': Precondition 'crime_occurred' not satisfied

❌ Invalid sequence: False
Violations: ["crime: Precondition 'protagonist_introduced' not satisfied", "investigation: Precondition 'crime_occurred' not satisfied"]


Merge_2 and 3


In [ ]:
# ==================== INTEGRATED PIPELINE ====================
# This combines Phase 1 (Neural Generation) + Phase 2 (Aesthetics) + Phase 3 (Planning)
# This is your COMPLETE research contribution!

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Tuple
import json

# Import previous modules (assumes they're loaded)
# from aesthetic_metrics import NarrativeAestheticEvaluator
# from planning_constraints import NarrativePlanner, generate_constrained_outline

# ==================== HYBRID STORY GENERATOR ====================
class HybridNarrativeGenerator:
    """
    Complete system bridging planning, aesthetics, and neural generation.
    This is what you'll showcase in your research!
    """

    def __init__(self, model_path: str, device: str = "auto"):
        """
        Initialize the hybrid system.

        Args:
            model_path: Path to your fine-tuned outline-to-story model
            device: 'cuda', 'cpu', or 'auto'
        """
        print("🚀 Initializing Hybrid Narrative Generator...")

        # Load trained model
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map=device,
            torch_dtype=torch.bfloat16
        )
        self.model.eval()

        # Initialize aesthetic evaluator (Phase 2)
        self.aesthetic_evaluator = NarrativeAestheticEvaluator()

        # Planning templates (Phase 3)
        self.planners = {
            'mystery': create_mystery_template(),
        }

        print("✅ System ready!")

    def generate_with_constraints(
        self,
        story_type: str = "mystery",
        outline_length: int = 5,
        target_aesthetic_score: float = 7.0,
        max_attempts: int = 3,
        **generation_kwargs
    ) -> Dict:
        """
        Generate story with planning constraints AND aesthetic objectives.

        This is the core innovation: iterative generate-evaluate-refine loop.

        Args:
            story_type: 'mystery' or 'hero_journey'
            outline_length: Number of events in outline
            target_aesthetic_score: Minimum acceptable aesthetic score
            max_attempts: How many generation attempts before giving up
            **generation_kwargs: Parameters for model.generate()

        Returns:
            Dict with story, outline, aesthetic scores, and metadata
        """
        print("\n" + "="*70)
        print(f"🎭 HYBRID GENERATION: {story_type.upper()}")
        print("="*70)

        # PHASE 3: Generate structurally valid outline
        print("\n📋 Step 1: Planning with constraints...")
        planner = self.planners.get(story_type)
        if not planner:
            raise ValueError(f"Unknown story type: {story_type}")

        outline_events = planner.generate_outline(target_length=outline_length)
        outline_string = planner.to_outline_string()

        print(f"✅ Valid outline: {outline_string}")

        # PHASE 1: Neural generation loop with aesthetic monitoring
        best_story = None
        best_score = 0
        all_attempts = []

        for attempt in range(max_attempts):
            print(f"\n📝 Step 2.{attempt+1}: Generating story from outline...")

            # Format prompt
            prompt = f"[OUTLINE] {outline_string} [/OUTLINE]\n[STORY]"

            # Generate
            story = self._generate_story(prompt, **generation_kwargs)

            # PHASE 2: Evaluate aesthetics
            print(f"\n📊 Step 3.{attempt+1}: Evaluating aesthetics...")
            aesthetic_results = self.aesthetic_evaluator.evaluate_story(story)
            score = aesthetic_results['overall_aesthetic_score']

            print(f"   Score: {score:.2f}/10 (target: {target_aesthetic_score})")

            all_attempts.append({
                'story': story,
                'score': score,
                'aesthetics': aesthetic_results
            })

            # Check if target reached
            if score >= target_aesthetic_score:
                print(f"✅ Target aesthetic score achieved!")
                best_story = story
                best_score = score
                break

            # Track best so far
            if score > best_score:
                best_story = story
                best_score = score

            print(f"   Best so far: {best_score:.2f}/10")

        # Return best result
        final_result = {
            'story': best_story,
            'outline': outline_string,
            'outline_events': [e.description for e in outline_events],
            'aesthetic_score': best_score,
            'aesthetic_details': self.aesthetic_evaluator.evaluate_story(best_story),
            'attempts': len(all_attempts),
            'all_attempts': all_attempts,
            'constraints_satisfied': True,  # Planning guarantees this
            'story_type': story_type
        }

        self._print_final_report(final_result)

        return final_result

    def _generate_story(self, prompt: str, **kwargs) -> str:
        """Internal method to generate story from outline"""
        # Default generation params (can be overridden)
        default_kwargs = {
            'max_new_tokens': 500,
            'temperature': 0.8,
            'top_p': 0.9,
            'do_sample': True,
            'pad_token_id': self.tokenizer.eos_token_id,
            'repetition_penalty': 1.2
        }
        default_kwargs.update(kwargs)

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(**inputs, **default_kwargs)

        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract story part (after [STORY] tag)
        if "[STORY]" in generated_text:
            story = generated_text.split("[STORY]")[-1].strip()
        else:
            story = generated_text

        return story

    def _print_final_report(self, result: Dict):
        """Pretty print final results"""
        print("\n" + "="*70)
        print("🎉 GENERATION COMPLETE - FINAL REPORT")
        print("="*70)

        print(f"\n📖 Story Type: {result['story_type']}")
        print(f"📋 Outline: {result['outline']}")
        print(f"\n🎯 Aesthetic Score: {result['aesthetic_score']:.2f}/10")

        aesthetics = result['aesthetic_details']['summary']
        print(f"\n📊 Aesthetic Breakdown:")
        print(f"   - Emotional Arc: {aesthetics['arc_type']}")
        print(f"   - Peak Suspense: {aesthetics['peak_suspense']:.2f}/10")
        print(f"   - Readability: {aesthetics['readability']:.1f}")
        print(f"   - Coherence: {aesthetics['coherence']:.2f}/10")

        print(f"\n🔄 Generation Attempts: {result['attempts']}")
        print(f"✅ Structural Constraints: Satisfied")

        print(f"\n📝 Generated Story Preview:")
        print("-" * 70)
        preview = result['story'][:400] + "..." if len(result['story']) > 400 else result['story']
        print(preview)
        print("="*70)

    def compare_approaches(self, story_type: str = "mystery") -> Dict:
        """
        Compare different generation strategies:
        1. Unconstrained generation (baseline)
        2. Planning only (no aesthetic optimization)
        3. Hybrid (planning + aesthetics) - YOUR APPROACH
        """
        print("\n" + "="*70)
        print("🔬 COMPARATIVE EVALUATION")
        print("="*70)

        results = {}

        # Baseline: No constraints
        print("\n1️⃣ Baseline: Unconstrained generation")
        baseline_prompt = "[STORY] A mysterious story begins"
        baseline_story = self._generate_story(baseline_prompt, max_new_tokens=300)
        baseline_aesthetics = self.aesthetic_evaluator.evaluate_story(baseline_story)
        results['baseline'] = {
            'story': baseline_story,
            'aesthetic_score': baseline_aesthetics['overall_aesthetic_score'],
            'has_constraints': False
        }
        print(f"   Score: {results['baseline']['aesthetic_score']:.2f}/10")

        # Planning only
        print("\n2️⃣ Planning Only: No aesthetic optimization")
        outline = generate_constrained_outline(story_type, length=3)
        planning_prompt = f"[OUTLINE] {outline} [/OUTLINE]\n[STORY]"
        planning_story = self._generate_story(planning_prompt, max_new_tokens=300)
        planning_aesthetics = self.aesthetic_evaluator.evaluate_story(planning_story)
        results['planning_only'] = {
            'story': planning_story,
            'aesthetic_score': planning_aesthetics['overall_aesthetic_score'],
            'has_constraints': True
        }
        print(f"   Score: {results['planning_only']['aesthetic_score']:.2f}/10")

        # Hybrid (YOUR APPROACH)
        print("\n3️⃣ Hybrid: Planning + Aesthetic Optimization")
        hybrid_result = self.generate_with_constraints(
            story_type=story_type,
            outline_length=3,
            target_aesthetic_score=7.0,
            max_attempts=2,
            max_new_tokens=300
        )
        results['hybrid'] = {
            'story': hybrid_result['story'],
            'aesthetic_score': hybrid_result['aesthetic_score'],
            'has_constraints': True,
            'attempts': hybrid_result['attempts']
        }

        # Summary comparison
        print("\n" + "="*70)
        print("📊 COMPARISON SUMMARY")
        print("="*70)
        print(f"{'Approach':<25} {'Aesthetic Score':<20} {'Constraints':<15}")
        print("-"*70)
        for name, data in results.items():
            constraints = "✅" if data['has_constraints'] else "❌"
            print(f"{name:<25} {data['aesthetic_score']:>6.2f}/10           {constraints:<15}")

        return results

    def save_results(self, result: Dict, filename: str):
        """Save generation results to JSON"""
        # Remove large objects for serialization
        save_data = {
            'story': result['story'],
            'outline': result['outline'],
            'outline_events': result['outline_events'],
            'aesthetic_score': result['aesthetic_score'],
            'aesthetic_summary': result['aesthetic_details']['summary'],
            'attempts': result['attempts'],
            'story_type': result['story_type']
        }

        with open(filename, 'w') as f:
            json.dump(save_data, f, indent=2)

        print(f"💾 Results saved to {filename}")

# ==================== USAGE EXAMPLES ====================
if __name__ == "__main__":
    # Initialize system with your trained model
    generator = HybridNarrativeGenerator(
        model_path="llama32_1b_outline_trained",  # Your Phase 1 model
        device="auto"
    )

    # Example 1: Generate mystery story with aesthetic optimization
    result = generator.generate_with_constraints(
        story_type="mystery",
        outline_length=5,
        target_aesthetic_score=7.0,
        max_attempts=3,
        max_new_tokens=400,
        temperature=0.9
    )

    # Save results
    generator.save_results(result, "mystery_result.json")

    # Example 2: Comparative evaluation (for your paper/presentation)
    comparison = generator.compare_approaches(story_type="mystery")

    print("\n✅ All examples completed!")

🚀 Initializing Hybrid Narrative Generator...


Device set to use cpu


✅ System ready!

🎭 HYBRID GENERATION: MYSTERY

📋 Step 1: Planning with constraints...
✅ Executed: intro (introduction)
⚠️ No valid events available, stopping early
✅ Valid outline: detective, quiet town, morning

📝 Step 2.1: Generating story from outline...

📊 Step 3.1: Evaluating aesthetics...
📊 Analyzing narrative aesthetics...
   Score: 5.04/10 (target: 7.0)
   Best so far: 5.04/10

📝 Step 2.2: Generating story from outline...

📊 Step 3.2: Evaluating aesthetics...
📊 Analyzing narrative aesthetics...
   Score: 2.51/10 (target: 7.0)
   Best so far: 5.04/10

📝 Step 2.3: Generating story from outline...

📊 Step 3.3: Evaluating aesthetics...
📊 Analyzing narrative aesthetics...
   Score: 2.58/10 (target: 7.0)
   Best so far: 5.04/10
📊 Analyzing narrative aesthetics...

🎉 GENERATION COMPLETE - FINAL REPORT

📖 Story Type: mystery
📋 Outline: detective, quiet town, morning

🎯 Aesthetic Score: 5.04/10

📊 Aesthetic Breakdown:
   - Emotional Arc: too_short
   - Peak Suspense: 1.07/10
   - Readab

In [ ]:
# ==================== INTEGRATED PIPELINE ====================
# This combines Phase 1 (Neural Generation) + Phase 2 (Aesthetics) + Phase 3 (Planning)
# This is your COMPLETE research contribution!

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Tuple
import json

# Import previous modules (assumes they're loaded)
# from aesthetic_metrics import NarrativeAestheticEvaluator
# from planning_constraints import NarrativePlanner, generate_constrained_outline

# ==================== HYBRID STORY GENERATOR ====================
class HybridNarrativeGenerator:
    """
    Complete system bridging planning, aesthetics, and neural generation.
    This is what you'll showcase in your research!
    """

    def __init__(self, model=None, tokenizer=None, model_path: str = None, device: str = "auto"):
        """
        Initialize the hybrid system.

        Args:
            model: Pre-loaded model (preferred - avoids reloading)
            tokenizer: Pre-loaded tokenizer (preferred)
            model_path: Path to load model from (only if model=None)
            device: 'cuda', 'cpu', or 'auto' (only used if loading from path)
        """
        print("🚀 Initializing Hybrid Narrative Generator...")

        # Load trained model
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map=device,
            torch_dtype=torch.bfloat16
        )
        self.model.eval()

        # Initialize aesthetic evaluator (Phase 2)
        self.aesthetic_evaluator = NarrativeAestheticEvaluator()

        # Planning templates (Phase 3)
        self.planners = {
            'mystery': create_mystery_template,
            'romance': create_romance_template,
            'horror': create_horror_template,
            'scifi': create_scifi_template,
            'adventure': create_adventure_template
        }

        print("✅ System ready!")

    def generate_with_constraints(
        self,
        story_type: str = "mystery",
        outline_length: int = 5,
        custom_outline: str = None,
        target_aesthetic_score: float = 7.0,
        max_attempts: int = 3,
        **generation_kwargs
    ) -> Dict:
        """
        Generate story with planning constraints AND aesthetic objectives.

        This is the core innovation: iterative generate-evaluate-refine loop.

        Args:
            story_type: 'mystery', 'hero_journey', 'romance', 'horror', 'scifi', 'adventure'
            outline_length: Number of events in outline
            custom_outline: Optional - provide your own outline string (bypasses planning)
            target_aesthetic_score: Minimum acceptable aesthetic score
            max_attempts: How many generation attempts before giving up
            **generation_kwargs: Parameters for model.generate()

        Returns:
            Dict with story, outline, aesthetic scores, and metadata
        """
        print("\n" + "="*70)
        print(f"🎭 HYBRID GENERATION: {story_type.upper() if not custom_outline else 'CUSTOM'}")
        print("="*70)

        # PHASE 3: Generate structurally valid outline
        if custom_outline:
            # User provided their own outline
            print("\n📋 Step 1: Using custom outline...")
            outline_string = custom_outline
            outline_events = []  # No planning validation for custom
        else:
            # Use planning template
            print("\n📋 Step 1: Planning with constraints...")
            planner_func = self.planners.get(story_type)
            if not planner_func:
                available = ", ".join(self.planners.keys())
                raise ValueError(f"Unknown story type: {story_type}. Available: {available}")

            planner = planner_func()
            outline_events = planner.generate_outline(target_length=outline_length)
            outline_string = planner.to_outline_string()

        print(f"✅ Outline: {outline_string}")

        # PHASE 1: Neural generation loop with aesthetic monitoring
        best_story = None
        best_score = 0
        all_attempts = []

        for attempt in range(max_attempts):
            print(f"\n📝 Step 2.{attempt+1}: Generating story from outline...")

            # Format prompt
            prompt = f"[OUTLINE] {outline_string} [/OUTLINE]\n[STORY]"

            # Generate
            story = self._generate_story(prompt, **generation_kwargs)

            # PHASE 2: Evaluate aesthetics
            print(f"\n📊 Step 3.{attempt+1}: Evaluating aesthetics...")
            aesthetic_results = self.aesthetic_evaluator.evaluate_story(story)
            score = aesthetic_results['overall_aesthetic_score']

            print(f"   Score: {score:.2f}/10 (target: {target_aesthetic_score})")

            all_attempts.append({
                'story': story,
                'score': score,
                'aesthetics': aesthetic_results
            })

            # Check if target reached
            if score >= target_aesthetic_score:
                print(f"✅ Target aesthetic score achieved!")
                best_story = story
                best_score = score
                break

            # Track best so far
            if score > best_score:
                best_story = story
                best_score = score

            print(f"   Best so far: {best_score:.2f}/10")

        # Return best result
        final_result = {
            'story': best_story,
            'outline': outline_string,
            'outline_events': [e.description for e in outline_events],
            'aesthetic_score': best_score,
            'aesthetic_details': self.aesthetic_evaluator.evaluate_story(best_story),
            'attempts': len(all_attempts),
            'all_attempts': all_attempts,
            'constraints_satisfied': True,  # Planning guarantees this
            'story_type': story_type
        }

        self._print_final_report(final_result)

        return final_result

    def _generate_story(self, prompt: str, **kwargs) -> str:
        """Internal method to generate story from outline"""
        # Default generation params (can be overridden)
        default_kwargs = {
            'max_new_tokens': 500,
            'temperature': 0.8,
            'top_p': 0.9,
            'do_sample': True,
            'pad_token_id': self.tokenizer.eos_token_id,
            'repetition_penalty': 1.2
        }
        default_kwargs.update(kwargs)

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(**inputs, **default_kwargs)

        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract story part (after [STORY] tag)
        if "[STORY]" in generated_text:
            story = generated_text.split("[STORY]")[-1].strip()
        else:
            story = generated_text

        return story

    def _print_final_report(self, result: Dict):
        """Pretty print final results"""
        print("\n" + "="*70)
        print("🎉 GENERATION COMPLETE - FINAL REPORT")
        print("="*70)

        print(f"\n📖 Story Type: {result['story_type']}")
        print(f"📋 Outline: {result['outline']}")
        print(f"\n🎯 Aesthetic Score: {result['aesthetic_score']:.2f}/10")

        aesthetics = result['aesthetic_details']['summary']
        print(f"\n📊 Aesthetic Breakdown:")
        print(f"   - Emotional Arc: {aesthetics['arc_type']}")
        print(f"   - Peak Suspense: {aesthetics['peak_suspense']:.2f}/10")
        print(f"   - Readability: {aesthetics['readability']:.1f}")
        print(f"   - Coherence: {aesthetics['coherence']:.2f}/10")

        print(f"\n🔄 Generation Attempts: {result['attempts']}")
        print(f"✅ Structural Constraints: Satisfied")

        print(f"\n📝 Generated Story Preview:")
        print("-" * 70)
        preview = result['story'][:400] + "..." if len(result['story']) > 400 else result['story']
        print(preview)
        print("="*70)

    def compare_approaches(self, story_type: str = "mystery") -> Dict:
        """
        Compare different generation strategies:
        1. Unconstrained generation (baseline)
        2. Planning only (no aesthetic optimization)
        3. Hybrid (planning + aesthetics) - YOUR APPROACH
        """
        print("\n" + "="*70)
        print("🔬 COMPARATIVE EVALUATION")
        print("="*70)

        results = {}

        # Baseline: No constraints
        print("\n1️⃣ Baseline: Unconstrained generation")
        baseline_prompt = "[STORY] A mysterious story begins"
        baseline_story = self._generate_story(baseline_prompt, max_new_tokens=300)
        baseline_aesthetics = self.aesthetic_evaluator.evaluate_story(baseline_story)
        results['baseline'] = {
            'story': baseline_story,
            'aesthetic_score': baseline_aesthetics['overall_aesthetic_score'],
            'has_constraints': False
        }
        print(f"   Score: {results['baseline']['aesthetic_score']:.2f}/10")

        # Planning only
        print("\n2️⃣ Planning Only: No aesthetic optimization")
        outline = generate_constrained_outline(story_type, length=3)
        planning_prompt = f"[OUTLINE] {outline} [/OUTLINE]\n[STORY]"
        planning_story = self._generate_story(planning_prompt, max_new_tokens=300)
        planning_aesthetics = self.aesthetic_evaluator.evaluate_story(planning_story)
        results['planning_only'] = {
            'story': planning_story,
            'aesthetic_score': planning_aesthetics['overall_aesthetic_score'],
            'has_constraints': True
        }
        print(f"   Score: {results['planning_only']['aesthetic_score']:.2f}/10")

        # Hybrid (YOUR APPROACH)
        print("\n3️⃣ Hybrid: Planning + Aesthetic Optimization")
        hybrid_result = self.generate_with_constraints(
            story_type=story_type,
            outline_length=3,
            target_aesthetic_score=7.0,
            max_attempts=2,
            max_new_tokens=300
        )
        results['hybrid'] = {
            'story': hybrid_result['story'],
            'aesthetic_score': hybrid_result['aesthetic_score'],
            'has_constraints': True,
            'attempts': hybrid_result['attempts']
        }

        # Summary comparison
        print("\n" + "="*70)
        print("📊 COMPARISON SUMMARY")
        print("="*70)
        print(f"{'Approach':<25} {'Aesthetic Score':<20} {'Constraints':<15}")
        print("-"*70)
        for name, data in results.items():
            constraints = "✅" if data['has_constraints'] else "❌"
            print(f"{name:<25} {data['aesthetic_score']:>6.2f}/10           {constraints:<15}")

        return results

    def save_results(self, result: Dict, filename: str):
        """Save generation results to JSON"""
        # Remove large objects for serialization
        save_data = {
            'story': result['story'],
            'outline': result['outline'],
            'outline_events': result['outline_events'],
            'aesthetic_score': result['aesthetic_score'],
            'aesthetic_summary': result['aesthetic_details']['summary'],
            'attempts': result['attempts'],
            'story_type': result['story_type']
        }

        with open(filename, 'w') as f:
            json.dump(save_data, f, indent=2)

        print(f"💾 Results saved to {filename}")

if __name__ == "__main__":
    # Initialize system with your trained model
    generator = HybridNarrativeGenerator(
        model_path="llama32_1b_outline_trained",  # Your Phase 1 model
        device="auto"
    )

    # Example 1: Generate mystery story with aesthetic optimization
    result = generator.generate_with_constraints(
        story_type="romance",
        outline_length=5,
        target_aesthetic_score=7.0,
        max_attempts=3,
        max_new_tokens=400,
        temperature=0.9
    )

    # Save results
    generator.save_results(result, "romance_story_result.json")

    # Example 2: Comparative evaluation (for your paper/presentation)
    comparison = generator.compare_approaches(story_type="romance")

    print("\n✅ All examples completed!")

🚀 Initializing Hybrid Narrative Generator...


Device set to use cpu


✅ System ready!

🎭 HYBRID GENERATION: ROMANCE

📋 Step 1: Planning with constraints...
✅ Executed: meet_cute (introduction)
✅ Executed: connection (conflict)
✅ Executed: obstacle (escalation)
✅ Executed: confession (climax)
✅ Executed: together (resolution)
✅ Outline: encounter, attraction, first meeting | chemistry, conversation, bonding | misunderstanding, conflict, barrier | truth, feelings, vulnerable | love, commitment, future

📝 Step 2.1: Generating story from outline...

📊 Step 3.1: Evaluating aesthetics...
📊 Analyzing narrative aesthetics...
   Score: 2.59/10 (target: 7.0)
   Best so far: 2.59/10

📝 Step 2.2: Generating story from outline...

📊 Step 3.2: Evaluating aesthetics...
📊 Analyzing narrative aesthetics...
   Score: 2.51/10 (target: 7.0)
   Best so far: 2.59/10

📝 Step 2.3: Generating story from outline...

📊 Step 3.3: Evaluating aesthetics...
📊 Analyzing narrative aesthetics...
   Score: 2.53/10 (target: 7.0)
   Best so far: 2.59/10
📊 Analyzing narrative aesthetics...

